# Libraries

In [3]:
import os
import torch
from PIL import Image
import matplotlib.pyplot as plt

import mySecrets

# Download

Run the code below to download the Banana dataset from Roboflow. 

After running the code below, edit the "data.yaml" file to contain the following:

```text
train: c:/Users/evgen/Desktop/230506_YOLO_banana/Banana-detection-3/train/images
val: c:/Users/evgen/Desktop/230506_YOLO_banana/Banana-detection-3/valid/images
test: c:/Users/evgen/Desktop/230506_YOLO_banana/Banana-detection-3/test/images

nc: 1
names: ['banana']
```


In [10]:
### Download the Banana dataset

!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key=mySecrets.roboflow_apiKey)
project = rf.workspace("evgenii-zorin-cm5us").project("banana-detection-7jjzn")
dataset = project.version(3).download("yolov5")



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to Banana-detection-3 in yolov5pytorch:: 100%|██████████| 314/314 [00:00<00:00, 1113.78it/s]


Subsequently, clone the YOLOv5 Github repository:

In [12]:
print( os.getcwd() )

### Download YOLOv5 Github repo
if 'yolov5' not in os.listdir():
	!git clone https://github.com/ultralytics/yolov5
	print(" > Successfully cloned github repo of YOLOv5!")
else:
	print(" > YOLOv5 repo already present.")

if os.path.basename(os.getcwd()) != 'yolov5':
	%cd yolov5

print(f" > current dir: {os.getcwd()}")
%pip install -qr requirements.txt
%pip install -q roboflow


c:\Users\evgen\Desktop\230506_YOLO_banana
 > YOLOv5 repo already present.
c:\Users\evgen\Desktop\230506_YOLO_banana\yolov5
 > current dir: c:\Users\evgen\Desktop\230506_YOLO_banana\yolov5
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


# Experiments

Subsequently, I ran some experiments on training YOLO based on different parameters:
- NANO weights: `yolov5n.pt` 
- 100 epochs train only vs 10x10 evolve -> 100 train epochs
- Fewer images vs more images with augmentation. At first, my dataset only had 44 images. When I saw that the performance of the trained model was too low, I decided to add more images (around 10) and perform augmentation on Roboflow, which resulted in the final dataset having around 150 images. 

Example commands that could be run:
```py
### train only
!python train.py --img 416 --batch 1 --epochs 100 --data ../Banana-detection-3/data.yaml --weights yolov5n.pt

### evolve first, then train with hyperparameters identified with evolve
!python train.py --img 416 --batch 1 --epochs 10 --data ../Banana-detection-3/data.yaml --weights yolov5n.pt --evolve 10
!python train.py --img 416 --batch 1 --epochs 100 --data ../Banana-detection-3/data.yaml --weights yolov5n.pt --hyp runs/evolve/exp/hyp_evolve.yaml
```

| Command | Parameters | P | R | mAP50 |
| - | - | - | - | - |
| 44 images -> train (100 epochs) | `--img 416` | 0.536 | 0.75 | 0.348 |
| 44 images -> Evolve (10 epochs * 10 evolve) -> train (100 epochs) | `--img 416` | 0.615 | 0.5 | 0.691 |
| 151 images -> train (100 epochs) | `--img 416` | 0.778 | 0.833 | 0.812 |
| 151 images -> Evolve (10 epochs * 10 evolve) -> train (100 epochs) | `--img 416` | 0.986 | 0.833 | 0.843 |

As per the experiments above, the best weights were saved in `Final_model/best_params/best.pt`. 


# Test

The final model could be run as:
```py
### Test images from the original dataset, test subset
!python val.py --task test --weights ../Final_model/best_params/best.pt --img 416 --data ../Banana-detection-3/data.yaml
### Test video coming from web camera
!python detect.py --weights ../Final_model/best_params/best.pt --source 0
```

The results of the object detection on the test portion of the dataset can be seen in `Predictions`:

<img src="Final_model/predictions/val_batch0_pred.jpg">
